In [1]:
import pandas as pd

import time
from glob import glob
import shutil
import logging

from vnstock3 import Vnstock

In [8]:
stock = Vnstock().stock(symbol='VNINDEX', source='vci')

2024-10-23 15:44:08,706 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [2]:
pd.read_csv("../data/gics/gics.csv")

,STT,Mã CK▲,Tên công ty,Ngành,Sàn,Khối lượng NY/ĐKGD
0,1,A32,CTCP 32,Hàng tiêu dùng và trang trí,UPCoM,6800000
1,2,AAA,CTCP Nhựa An Phát Xanh,Nguyên vật liệu,HOSE,382274496
2,3,AAH,CTCP Hợp Nhất,Năng lượng,UPCoM,117900000
3,4,AAM,CTCP Thủy sản MeKong,"Thực phẩm, đồ uống và thuốc lá",HOSE,12346411
4,5,AAS,CTCP Chứng khoán SmartInvest,Dịch vụ tài chính,UPCoM,229999521
...,...,...,...,...,...,...
3270,3271,YenBai,CTCP Xây dựng Số 3 Yên Bái,NaN,OTC,0
3271,3272,YRC,CTCP Đường sắt Yên Lào,Vận tải,OTC,1900000
3272,3273,YSC,CTCP Hapaco Yên Sơn,Nguyên vật liệu,OTC,1100000
3273,3274,YSVN,Công ty TNHH Chứng khoán Yuanta Việt Nam,Dịch vụ tài chính,Khác,30000000
